In [1]:
import pandas as pd
import requests
import yfinance as yf
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/var/folders/pf/fm0v0g_52r754cfv3r4lqv980000gn/T/ipykernel_55059/3286336062.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def idv_stock_rec(stock_symbol: str) -> dict:
    """
    Get stock data from Alpha Vantage API
    request arguments: the symbol of the stock
    returns: the in depth information of the stock including:
        name and other basic stock information like data,
        strongestPrinciple and source/reasoning for strongest principle as a short blurb,
        weakestPrinciple and source/reasoning for weakest principle as a short blurb,
        relevant news articles, 
    """

    headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'}

    stock = yf.Ticker(stock_symbol)
    info = stock.info
    
    url = "https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={}".format(stock_symbol)
    response = requests.get(url, headers=headers)
    esg_data = response.json()

    if esg_data['esgChart']['result'] == [{}]:
        esg_score = 'N/A'
        env_score = 'N/A'
        gov_score = 'N/A'
        soc_score = 'N/A'
    else:
        esg_score = esg_data['esgChart']['result'][0]['symbolSeries']['esgScore'][0]
        env_score = esg_data['esgChart']['result'][0]['symbolSeries']['environmentScore'][0]
        gov_score = esg_data['esgChart']['result'][0]['symbolSeries']['governanceScore'][0]
        soc_score = esg_data['esgChart']['result'][0]['symbolSeries']['socialScore'][0]

    scores = {env_score: 'environmentScore', gov_score: 'governanceScore', soc_score: 'socialScore'}
    min_score = scores[min(scores.keys())]
    max_score = scores[max(scores.keys())]

    url = 'https://ca.finance.yahoo.com/quote/AAPL/sustainability'
    involvements = pd.read_html(url, storage_options=headers, header=None)
    involvements = involvements[1]

    for i in range(len(involvements)):
        info[involvements.iloc[i][0]] = involvements.iloc[i][1]

    info['esg_score'] = esg_score
    info['env_score'] = env_score
    info['gov_score'] = gov_score
    info['soc_score'] = soc_score
    info['strongest_principle'] = max_score
    info['weakestPrinciple'] = min_score

    return info

In [5]:
def stock_rec(selected_filters: list) -> dict:
    """
    Get stock recommendation from Sovestico API
    //request arguments: a list of principles to filter by
    // returns: paginated data of top suggestions
    {
        suggestions:
            [
                {
                    symbol: "",
                    name: "",
                    price: 0.0,
                    change: -0.07,
                    strongestPrinciple: "",
                    weakestPrinciple: "",
                    esg: 0,
                    stockData: [//dependent on what API we decide to use]
                },
                ...
            ]
    }
    """
    tickers = pd.read_csv('SP500.csv')['Symbol']

    stock_df = pd.DataFrame()
    for ticker in tickers:
        stock = pd.DataFrame.from_dict([idv_stock_rec(ticker)],orient='columns')
        stock_df = pd.concat([stock_df, stock], ignore_index=True)
    return stock_df

In [7]:
stock_rec([])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)